In [ ]:
!pip install llmlingua llama-index llama-index-postprocessor-longllmlingua

In [ ]:
# OpenAI 키 설정
import os
os.environ["OPENAI_API_KEY"] = ''

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)
Settings.llm= OpenAI(model='gpt-4o-mini')

In [ ]:
# 폴그라함 에쎼이 다큐먼트화
documents = SimpleDirectoryReader("/content/pg").load_data()

# 인덱스만들기
index = VectorStoreIndex.from_documents(documents=documents)

# 시연 목적에 따라 long context retrieve 하고자 함
retriever = index.as_retriever()

In [ ]:
question = "Where did the author go for art school?"

In [ ]:
contexts =

In [ ]:
context_list = [n.get_content() for n in contexts]

In [ ]:
context_list

In [ ]:
#query_engine대신 vanilla하게 llm completion 호출
llm = OpenAI(model="gpt-4o-mini")
prompt = "\n\n".join(context_list + [question])

response = llm.complete(prompt)
print(str(response))

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers import CompactAndRefine
from llama_index.postprocessor.longllmlingua import LongLLMLinguaPostprocessor

# Node Postprocessor : retrieve 이후 <-> generation 전. 즉, Augmentation 스테이지에서의 프로세서.
node_postprocessor = LongLLMLinguaPostprocessor(
    instruction_str="Given the context, please answer the final question",
    target_token=300, # 목표로 하는 압축 토큰 수준
    rank_method="longllmlingua",
    additional_compress_kwargs={
        "condition_compare": True, # Compress 작업 시 오리지널 쿼리를 고려할 것인지에 대한 세팅
        "condition_in_question": "after", # 고려 시 배치
        "context_budget": "+100", #default값 사용 추천
        "reorder_context": "sort",
        "dynamic_context_compression_ratio": 0.3, #default값 사용 추천
    },
)

In [ ]:
# compression 작업 위해 retrieved_nodes에 컨텍스트 청크 저장
retrieved_nodes = retriever.retrieve(question)

# 추후 compressed context prompt 받을 시 response generation 방식 pre-define 해 놓은 것.
synthesizer = CompactAndRefine()

In [ ]:
from llama_index.core.schema import QueryBundle

# Compression 진행
new_retrieved_nodes = node_postprocessor.postprocess_nodes(
    retrieved_nodes, query_bundle=
)

In [ ]:
# compressed context 확인
new_retrieved_nodes

In [ ]:
original_contexts = "\n\n".join([n.get_content() for n in retrieved_nodes])
compressed_contexts = "\n\n".join([n.get_content() for n in new_retrieved_nodes])

# 기존의 토큰 수 카운트
original_tokens = node_postprocessor._llm_lingua.get_token_length(original_contexts)
# 컴프레스 완료된 토큰 수 카운트
compressed_tokens = node_postprocessor._llm_lingua.get_token_length(compressed_contexts)

print(compressed_contexts)
print()
print("Original Tokens:", original_tokens)
print("Compressed Tokens:", compressed_tokens)
print("Compressed Ratio:", f"{original_tokens/(compressed_tokens + 1e-5):.2f}x")

In [ ]:
# 최종적으로 답안 생성
response =

In [ ]:
print(str(response))

- 한방에 하는법

In [ ]:
# 처음부터 QueryEngine으로 retriever 연결, 사전 정의한 노드 포스트프로세서 오브젝트만 참조
retriever_query_engine = RetrieverQueryEngine.from_args(
    retriever,
)

In [ ]:
response =

In [ ]:
print(str(response))